<a href="https://colab.research.google.com/github/Mark1708/beautify-old-photo/blob/main/BeautifyOldPhoto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries used:

*   [Coloize](https://github.com/jantic/DeOldify)
*   [Restoration of faces](https://github.com/TencentARC/GFPGAN)



# Download libs and install requirements 
(Don't forget restart)

In [ ]:
# If the link does not work, you can download the archive from the link - https://drive.google.com/file/d/1O-neWA-asLI1c1-LTxdpTtzzmkMkw4cF/view?usp=sharing
!wget "https://www.dropbox.com/s/2yrzr98wj1faxi3/libs_beautify_photo.zip"
!unzip "/content/libs_beautify_photo.zip" && rm "/content/libs_beautify_photo.zip"
!rm -r "/content/__MACOSX"

In [ ]:
!pip install -r /content/colab_requirements.txt

# Import libs

In [ ]:
import sys
import os

sys.path.append('/content/libs/GFPGAN')
sys.path.append('/content/libs/DeOldify')

In [ ]:
import torch
import fastai
from deoldify import device
from deoldify.visualize import *
from deoldify.device_id import DeviceId
from pathlib import Path

# Prepare image folders
(Don't forget upload images in `input/` folder)

In [ ]:
!mkdir '/content/input' 
!mkdir '/content/high_resolution_output' 
!mkdir '/content/final_colorized_output'
!mkdir '/content/final_colorized_output/from_original' 
!mkdir '/content/final_colorized_output/from_recontraction'

# Improve quality

In [ ]:
!cd /content/libs/GFPGAN && python setup.py develop
!cd /content/libs/GFPGAN && python inference_gfpgan.py -i ../../input -o ../../high_resolution_output -v 1.3 -s 2

# Colorize images

In [ ]:
device.set(device=DeviceId.GPU0)
if not torch.cuda.is_available():
    print('GPU not available.')
torch.backends.cudnn.benchmark = True

In [ ]:
colorizer = get_image_colorizer(artistic=False)

In [ ]:
input = "Both" #@param ["Original images", "Images after Face Restoration", "Both"]
# result_dir_path = '/content/result_images' #@param {type:"string"}
render_factor = 17  #@param {type: "slider", min: 7, max: 40}
# watermarked = False #@param {type:"boolean"}
# display_render_factor = False #@param {type:"boolean"}
image_pathes = []
result_pathes = []
if input == "Original images":
  image_pathes.append('/content/input')
  result_pathes.append('/content/final_colorized_output/from_original')
if input == "Images after Face Restoration":
  image_pathes.append('/content/high_resolution_output/restored_imgs')
  result_pathes.append('/content/final_colorized_output/from_recontraction')
if input == "Both":
  image_pathes.append('/content/input')
  result_pathes.append('/content/final_colorized_output/from_original')
  image_pathes.append('/content/high_resolution_output/restored_imgs')
  result_pathes.append('/content/final_colorized_output/from_recontraction')

If you don't need to draw the result, you need to comment out lines 108-116 in the `plot_transformed_image` method (when there are too many photos, rendering slows down and causes a block overflow error)

In [ ]:
def colorize_photo(image_path, result_dir_path, render_factor, watermarked=False, display_render_factor=False):
  colorizer.plot_transformed_image(image_path, results_dir=Path(result_dir_path), render_factor=render_factor, display_render_factor=display_render_factor, figsize=(8,8), watermarked=watermarked)

In [ ]:
import imghdr

In [ ]:
for i in range(len(image_pathes)):
  # os.mkdir(result_pathes[i])
  for file in os.listdir(image_pathes[i]):
    full_path = image_pathes[i] + "/" + file
    if os.path.isfile(full_path) and imghdr.what(full_path) in ['jpg', 'png', 'jpeg']:
      colorize_photo(full_path, result_pathes[i], render_factor) 

# Ziped result

In [ ]:
!zip -r "colorized_output.zip" "/content/final_colorized_output" 